<a href="https://colab.research.google.com/github/FrancoPalavicinoG/cellia/blob/main/notebooks/02_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load DF from Google Drive

Mount Drive

In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import libraries

In [99]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Load Dataframes

In [100]:
input_path = "/content/drive/MyDrive/cellia_drive/Datasets/"

df_demo = pd.read_excel(input_path + "demographics.xlsx")
df_exam = pd.read_excel(input_path + "examination.xlsx")
df_lab = pd.read_excel(input_path + "labs.xlsx")
df_quest = pd.read_excel(input_path + "questionnaire.xlsx")

In [101]:
print("Demographics shape:", df_demo.shape)
print("Examination shape:", df_exam.shape)
print("Labs shape:", df_lab.shape)
print("Questionnaire shape:", df_quest.shape)

Demographics shape: (10175, 5)
Examination shape: (9813, 5)
Labs shape: (9813, 15)
Questionnaire shape: (10175, 14)


### DF examples

In [102]:
def data_example(df, name, n=5):
    print(f"\n{name} – Data examples:")
    display(df.head(n))

In [103]:
dfs = {
    "Demographic": df_demo,
    "Examination": df_exam,
    "Labs": df_lab,
    "Questionnaire": df_quest
}

In [104]:
for name, df in dfs.items():
    data_example(df, name)


Demographic – Data examples:


,SEQN,GENDER,AGE_MONTHS_1,AGE_YEARS,AGE_MONTHS_2
0,73557,1,NaN,69,NaN
1,73558,1,NaN,54,NaN
2,73559,1,NaN,72,NaN
3,73560,1,NaN,9,119.0
4,73561,2,NaN,73,NaN



Examination – Data examples:


,SEQN,WEIGHT_COMMENT,BODY_MASS_INDEX,HEIGHT,WIGHT
0,73557,NaN,26.7,171.3,78.3
1,73558,NaN,28.6,176.8,89.5
2,73559,NaN,28.9,175.3,88.9
3,73560,NaN,17.1,137.3,32.2
4,73561,NaN,19.7,162.4,52.0



Labs – Data examples:


,SEQN,TOTAL_CHOL_1,TOTAL_CHOL_2,CHOLESTEROL_1,CREATININE_1,TRIGLYCERIDES_R1,CHOLESTEROL_2,CREATININE_2,TRIGLYCERIDES_R2,LDL_1,LDL_2,TRIGLYCERIDE_1,TRIGLYCERIDE_2,HDL_1,HDL_2
0,73557,4.32,167.0,4.344,106.96,1.581,168.0,1.21,140.0,NaN,NaN,NaN,NaN,65.0,1.68
1,73558,4.40,170.0,4.319,69.84,2.902,167.0,0.79,257.0,NaN,NaN,NaN,NaN,50.0,1.29
2,73559,3.26,126.0,3.284,107.85,0.576,127.0,1.22,51.0,56.0,1.448,0.576,51.0,60.0,1.55
3,73560,4.34,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,1.58
4,73561,5.20,201.0,5.353,64.53,0.994,207.0,0.73,88.0,101.0,2.612,0.847,75.0,85.0,2.20



Questionnaire – Data examples:


,SEQN,HYPERTENSION_1,HYPERTENSION_2,HIGH_CHOLESTEROL,HEART_FAILURE,CORO_HEART_DISEASE,ANGINA_PECTORIS,HEART_ATTACK,HEART_FAILURE_AGE,CORO_HEART_DISEASE_AGE,ANGINA_PECTORIS_AGE,HEART_ATTACK_AGE,STROKE_AGE,HEART_ATTACK_RELATIVES
0,73557,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN,62.0,2.0
1,73558,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,2.0
2,73559,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,1.0
3,73560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,1.0


## ETL

### 1. Normalize Age, Sex, Weight, Height, and BMI

Map gender in binary form
*   1: Male -> 1
*   2: Female -> 0

In [105]:
gender_map = {1: 1, 2: 0}
df_demo['GENDER'] = df_demo['GENDER'].map(gender_map)

Drop rows where AGE_YEARS <= 18

In [106]:
df_demo = df_demo[df_demo['AGE_YEARS'] > 18].copy()

Drop column AGE_MONTHS_1 (Used with under two years old)

In [107]:
df_demo = df_demo.drop(columns=['AGE_MONTHS_1'])

Convert special values (7777 = refused, 9999 = don’t know) to NaN

In [108]:
df_demo['AGE_YEARS'] = df_demo['AGE_YEARS'].replace({7777: np.nan, 9999: np.nan})

Convert age in years to age in months.

In [109]:
def age_years_to_months(df, col_years='AGE_YEARS', col_months='AGE_MONTHS_2'):
    df[col_months] = df[col_years] * 12
    return df

In [110]:
df_demo = age_years_to_months(df_demo)

Rename AGE_MONTHS_2

In [111]:
df_demo = df_demo.rename(columns={'AGE_MONTHS_2': 'AGE_MONTHS'})

In [112]:
display(df_demo.head())

,SEQN,GENDER,AGE_YEARS,AGE_MONTHS
0,73557,1,69,828
1,73558,1,54,648
2,73559,1,72,864
4,73561,0,73,876
5,73562,1,56,672


In [113]:
def null_summary(df, name):
    nulls = df.isnull().sum()
    total = len(df)
    summary = pd.DataFrame({
        "Column": nulls.index,
        "Nulls": nulls.values,
        "Percent": (nulls.values / total * 100).round(2)
    })
    print(f"\nDataset: {name}")
    display(summary)

In [114]:
null_summary(df_demo, "Demographics")


Dataset: Demographics


,Column,Nulls,Percent
0,SEQN,0,0.0
1,GENDER,0,0.0
2,AGE_YEARS,0,0.0
3,AGE_MONTHS,0,0.0


Rename WEIGHT column

In [119]:
df_exam = df_exam.rename(columns={'WIGHT': 'WEIGHT'})

Drop the WEIGHT_COMMENT column

In [116]:
df_exam = df_exam.drop(columns=['WEIGHT_COMMENT'])

 Calculate BMI for rows with missing BMI, given weight (kg) and height (cm).

In [121]:
def calculate_bmi(df, weight_col='WEIGHT', height_col='HEIGHT', bmi_col='BODY_MASS_INDEX'):
    # Only calculate where BMI is missing and height/weight are present
    mask = df[bmi_col].isna() & df[weight_col].notna() & df[height_col].notna()

    # Convert height to meters
    height_m = df.loc[mask, height_col] / 100

    # Calculate BMI
    df.loc[mask, bmi_col] = df.loc[mask, weight_col] / (height_m ** 2)

    return df

Calculate weight where missing given BMI and height.

In [124]:
def calculate_weight_from_bmi(df, bmi_col='BODY_MASS_INDEX', height_col='HEIGHT', weight_col='WEIGHT'):
    mask = df[weight_col].isna() & df[bmi_col].notna() & df[height_col].notna()
    height_m = df.loc[mask, height_col] / 100
    df.loc[mask, weight_col] = df.loc[mask, bmi_col] * (height_m ** 2)
    return df

Calculate height where missing given BMI and weight.

In [128]:
def calculate_height_from_bmi(df, bmi_col='BODY_MASS_INDEX', weight_col='WEIGHT', height_col='HEIGHT'):
    mask = df[height_col].isna() & df[bmi_col].notna() & df[weight_col].notna()
    df.loc[mask, height_col] = np.sqrt(df.loc[mask, weight_col] / df.loc[mask, bmi_col]) * 100
    return df

In [129]:
df_exam = calculate_bmi(df_exam)

In [130]:
df_exam = calculate_weight_from_bmi(df_exam)
df_exam = calculate_height_from_bmi(df_exam)

Drop rows where HEIGHT or WEIGHT is missing

In [132]:
df_exam = df_exam[df_exam['HEIGHT'].notna() & df_exam['WEIGHT'].notna()].copy()

In [133]:
display(df_exam.head())
null_summary(df_exam, "Examination")

,SEQN,BODY_MASS_INDEX,HEIGHT,WEIGHT
0,73557,26.7,171.3,78.3
1,73558,28.6,176.8,89.5
2,73559,28.9,175.3,88.9
3,73560,17.1,137.3,32.2
4,73561,19.7,162.4,52.0



Dataset: Examination


,Column,Nulls,Percent
0,SEQN,0,0.0
1,BODY_MASS_INDEX,0,0.0
2,HEIGHT,0,0.0
3,WEIGHT,0,0.0
